# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument For Doc2Vec

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Doc2Vec Model

### Twitter

In [12]:
taggerName = 'ct'

#### News to tagged Document

In [13]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_doc2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_doc2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_doc2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_doc2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_doc2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_doc2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [14]:
model1 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
name1 = '-'.join(re.split('[\(\),\/]',str(model1)))+taggerName
model2 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
name2 = '-'.join(re.split('[\(\),\/]',str(model2)))+taggerName
model3 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')
name3 = '-'.join(re.split('[\(\),\/]',str(model3)))+taggerName

#### tagged Document to Vector

In [15]:
# 다음
daum_news_by_m1_name = newsPath+'daum_news_vector_'+name1
daum_news_by_m2_name = newsPath+'daum_news_vector_'+name2
daum_news_by_m3_name = newsPath+'daum_news_vector_'+name3
if not os.path.isfile(daum_news_by_m1_name):
    daum_news_by_m1 = bm.Get_Infer_Vector(taggedDaumData, model1)
    pickle.dump(daum_news_by_m1, open(daum_news_by_m1_name, 'wb'))
    del daum_news_by_m1

if not os.path.isfile(daum_news_by_m2_name):
    daum_news_by_m2 = bm.Get_Infer_Vector(taggedDaumData, model2)
    pickle.dump(daum_news_by_m2, open(daum_news_by_m2_name, 'wb'))
    del daum_news_by_m2
    
if not os.path.isfile(daum_news_by_m3_name):
    daum_news_by_m3 = bm.Get_Infer_Vector(taggedDaumData, model3)
    pickle.dump(daum_news_by_m3, open(daum_news_by_m3_name, 'wb'))
    del daum_news_by_m3

In [16]:
# 네이버
naver_news_by_m1_name = newsPath+'daum_news_vector_'+name1
naver_news_by_m2_name = newsPath+'daum_news_vector_'+name2
naver_news_by_m3_name = newsPath+'daum_news_vector_'+name3
if not os.path.isfile(naver_news_by_m1_name):
    naver_news_by_m1 = bm.Get_Infer_Vector(taggedDaumData, model1)
    pickle.dump(naver_news_by_m1, open(naver_news_by_m1_name, 'wb'))
    del naver_news_by_m1

if not os.path.isfile(naver_news_by_m2_name):
    naver_news_by_m2 = bm.Get_Infer_Vector(taggedDaumData, model2)
    pickle.dump(naver_news_by_m2, open(naver_news_by_m2_name, 'wb'))
    del naver_news_by_m2
    
if not os.path.isfile(naver_news_by_m3_name):
    naver_news_by_m3 = bm.Get_Infer_Vector(taggedDaumData, model3)
    pickle.dump(naver_news_by_m3, open(naver_news_by_m3_name, 'wb'))
    del naver_news_by_m3

In [ ]:
del taggedDaumData
del taggedNaverData

#### model1

In [17]:
classifierList = glob(classifierPath+'*'+name1)

In [18]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
RandomForestClassifier
NeuralNetwork_1
NeuralNetwork_2
XGBoost
SVC


In [30]:
%%time
daum_news_by_m1 = pickle.load(open(daum_news_by_m1_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m1, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name1,index=None, encoding='utf-8')

9372it [00:00, 813884.99it/s]
9372it [00:00, 741763.54it/s]
9372it [00:00, 662571.08it/s]


CPU times: user 31.3 s, sys: 574 ms, total: 31.9 s
Wall time: 32.3 s


In [31]:
%%time
naver_news_by_m1 = pickle.load(open(naver_news_by_m1_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m1, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name1,index=None, encoding='utf-8')

9372it [00:00, 726679.80it/s]
9372it [00:00, 874097.02it/s]
9372it [00:00, 863687.67it/s]


CPU times: user 31.4 s, sys: 586 ms, total: 32 s
Wall time: 32.8 s


In [ ]:
del daum_news_by_m1
del naver_news_by_m1
del daum_news_by_m1_name
del naver_news_by_m1_name
del name1
del model1
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver

#### model2

In [32]:
classifierList = glob(classifierPath+'*'+name2)

In [33]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [34]:
%%time
daum_news_by_m2 = pickle.load(open(daum_news_by_m2_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m2, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name2,index=None, encoding='utf-8')

9372it [00:00, 515934.07it/s]
9372it [00:00, 840008.06it/s]
9372it [00:00, 668520.70it/s]


CPU times: user 13.8 s, sys: 458 ms, total: 14.2 s
Wall time: 13.6 s


In [35]:
%%time
naver_news_by_m2 = pickle.load(open(naver_news_by_m2_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m2, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name2,index=None, encoding='utf-8')

9372it [00:00, 464212.11it/s]
9372it [00:00, 882416.71it/s]
9372it [00:00, 709152.23it/s]


CPU times: user 14.1 s, sys: 402 ms, total: 14.5 s
Wall time: 14.1 s


In [ ]:
del daum_news_by_m2
del naver_news_by_m2
del daum_news_by_m2_name
del naver_news_by_m2_name
del name2
del model2
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver

#### model3

In [36]:
classifierList = glob(classifierPath+'*'+name3)

In [37]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [38]:
%%time
daum_news_by_m3 = pickle.load(open(daum_news_by_m3_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m3, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name3,index=None, encoding='utf-8')

9372it [00:00, 473688.22it/s]
9372it [00:00, 828587.45it/s]
9372it [00:00, 822243.96it/s]


CPU times: user 35.3 s, sys: 873 ms, total: 36.1 s
Wall time: 36.3 s


In [39]:
%%time
naver_news_by_m3 = pickle.load(open(naver_news_by_m3_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m3, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name3,index=None, encoding='utf-8')

9372it [00:00, 714164.04it/s]
9372it [00:00, 740575.69it/s]
9372it [00:00, 725231.86it/s]


CPU times: user 35.5 s, sys: 604 ms, total: 36.1 s
Wall time: 36.2 s


In [ ]:
del daum_news_by_m3
del naver_news_by_m3
del daum_news_by_m3_name
del naver_news_by_m3_name
del name3
del model3
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver

### Mecab

In [9]:
taggerName = 'mecab'

#### News to tagged Document

In [10]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_doc2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_doc2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_doc2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_doc2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_doc2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_doc2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [ ]:
model1 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-mecab.model')
name1 = '-'.join(re.split('[\(\),\/]',str(model1)))+taggerName
model2 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-mecab.model')
name2 = '-'.join(re.split('[\(\),\/]',str(model2)))+taggerName
model3 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-mecab.model')
name3 = '-'.join(re.split('[\(\),\/]',str(model3)))+taggerName

#### tagged Document to Vector

In [ ]:
# 다음
daum_news_by_m1_name = newsPath+'daum_news_vector_'+name1
daum_news_by_m2_name = newsPath+'daum_news_vector_'+name2
daum_news_by_m3_name = newsPath+'daum_news_vector_'+name3
if not os.path.isfile(daum_news_by_m1_name):
    daum_news_by_m1 = bm.Get_Infer_Vector(taggedDaumData, model1)
    pickle.dump(daum_news_by_m1, open(daum_news_by_m1_name, 'wb'))
    del daum_news_by_m1

if not os.path.isfile(daum_news_by_m2_name):
    daum_news_by_m2 = bm.Get_Infer_Vector(taggedDaumData, model2)
    pickle.dump(daum_news_by_m2, open(daum_news_by_m2_name, 'wb'))
    del daum_news_by_m2
    
if not os.path.isfile(daum_news_by_m3_name):
    daum_news_by_m3 = bm.Get_Infer_Vector(taggedDaumData, model3)
    pickle.dump(daum_news_by_m3, open(daum_news_by_m3_name, 'wb'))
    del daum_news_by_m3

 30%|██▉       | 2811/9372 [1:25:01<3:18:27,  1.81s/it]

In [ ]:
# 네이버
naver_news_by_m1_name = newsPath+'daum_news_vector_'+name1
naver_news_by_m2_name = newsPath+'daum_news_vector_'+name2
naver_news_by_m3_name = newsPath+'daum_news_vector_'+name3
if not os.path.isfile(naver_news_by_m1_name):
    naver_news_by_m1 = bm.Get_Infer_Vector(taggedDaumData, model1)
    pickle.dump(naver_news_by_m1, open(naver_news_by_m1_name, 'wb'))
    del naver_news_by_m1

if not os.path.isfile(naver_news_by_m2_name):
    naver_news_by_m2 = bm.Get_Infer_Vector(taggedDaumData, model2)
    pickle.dump(naver_news_by_m2, open(naver_news_by_m2_name, 'wb'))
    del naver_news_by_m2
    
if not os.path.isfile(naver_news_by_m3_name):
    naver_news_by_m3 = bm.Get_Infer_Vector(taggedDaumData, model3)
    pickle.dump(naver_news_by_m3, open(naver_news_by_m3_name, 'wb'))
    del naver_news_by_m3

In [ ]:
del taggedDaumData
del taggedNaverData

#### model1

In [ ]:
classifierList = glob(classifierPath+'*'+name1)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
daum_news_by_m1 = pickle.load(open(daum_news_by_m1_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m1, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name1,index=None, encoding='utf-8')

In [ ]:
%%time
naver_news_by_m1 = pickle.load(open(naver_news_by_m1_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m1, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name1,index=None, encoding='utf-8')

In [ ]:
del daum_news_by_m1
del naver_news_by_m1
del daum_news_by_m1_name
del naver_news_by_m1_name
del name1
del model1
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver

#### model2

In [ ]:
classifierList = glob(classifierPath+'*'+name2)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
daum_news_by_m2 = pickle.load(open(daum_news_by_m2_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m2, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name2,index=None, encoding='utf-8')

In [ ]:
%%time
naver_news_by_m2 = pickle.load(open(naver_news_by_m2_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m2, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name2,index=None, encoding='utf-8')

In [ ]:
del daum_news_by_m2
del naver_news_by_m2
del daum_news_by_m2_name
del naver_news_by_m2_name
del name2
del model2
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver

#### model3

In [ ]:
classifierList = glob(classifierPath+'*'+name3)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
daum_news_by_m3 = pickle.load(open(daum_news_by_m3_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_news_by_m3, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+name3,index=None, encoding='utf-8')

In [ ]:
%%time
naver_news_by_m3 = pickle.load(open(naver_news_by_m3_name, 'rb'))
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_news_by_m3, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+name3,index=None, encoding='utf-8')

In [ ]:
del daum_news_by_m3
del naver_news_by_m3
del daum_news_by_m3_name
del naver_news_by_m3_name
del name3
del model3
del loadClassifierDict
del predictOutcome_daum
del predictOutcome_naver